# Extracting Velocity Based on Contrast Dispersion in Grafts

 - Extract the graft from the peak intensity image on SimVascular:

        Extract .pth of the manual annotation and .vtu of the mesh

 - Register the CT-MPI sequence if needed.
 - Project the Image values onto the mesh.
 - Convert pathline to vtp and project image onto the pathline.
 - Use a gradient filter and argmax to find the point of shuttle mode on the pathline.
 - Clip the lumen at the point where the shuttle mode causes inconsistency.
 - Take a cross-sectional sample every 5 mm along the lumen.
 - Extract the TAC on every point.
 - Detect the upslope. (?)
 - Interpolate each point.
 - Concatenate upper and lower parts of the lumen at the same time points.
 - Extract temporal and spatial gradient.
 - Extract the velocity.
 - Baysian Framework

In [15]:
import vtk
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from utilities import *


In [4]:
path_ = "/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/3_VesselProjection/VA08B"
graft_name = "LIMA"
vtu_file_ = os.path.join(path_,f"{graft_name}.vtu")
centerline_file_ = os.path.join(path_, f"{graft_name}.pth")
Image_directory_ = "/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA08B"
Image_names = sorted(glob.glob(f"{Image_directory_}/*.vtk"))

In [5]:
Image_names

['/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA08B/VA08B_01.vtk',
 '/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA08B/VA08B_02.vtk',
 '/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA08B/VA08B_03.vtk',
 '/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA08B/VA08B_04.vtk',
 '/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA08B/VA08B_05.vtk',
 '/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA08B/VA08B_06.vtk',
 '/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA08B/VA08B_07.vtk',
 '/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA08B/VA08B_08.vtk',
 '/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA08B/VA08B_09.vtk',
 '/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA08B/VA08B_10.vtk',
 '/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImage

### Project Images to the Mesh

In [7]:
def ProbeFilter(TargetData, SourceData):
    ProbeFilter=vtk.vtkProbeFilter()
    ProbeFilter.SetInputData(TargetData)
    ProbeFilter.SetSourceData(SourceData)
    ProbeFilter.Update()
    
    return ProbeFilter.GetOutput()

In [11]:
os.path.splitext(os.path.basename(Image_names[0]))[0]

'VA08B_01'

In [12]:
Mesh = ReadVTUFile(vtu_file_)
MeshProjections = {}
for image_name in Image_names:
    Image_ = ReadVTKFile(image_name)
    mesh_projection = ProbeFilter(Mesh, Image_)
    image_root_name = os.path.splitext(os.path.basename(image_name))[0]
    mesh_name = f"{graft_name}_{image_root_name}"
    MeshProjections[image_root_name] = mesh_projection
    WriteVTUFile(os.path.join(path_,f"{mesh_name}.vtu"), mesh_projection)

In [13]:
MeshProjections

{'VA08B_01': <vtkmodules.vtkCommonDataModel.vtkUnstructuredGrid(0x7f8b01023c00) at 0x1a55e3c40>,
 'VA08B_02': <vtkmodules.vtkCommonDataModel.vtkUnstructuredGrid(0x7f8b00729310) at 0x18aa5cdc0>,
 'VA08B_03': <vtkmodules.vtkCommonDataModel.vtkUnstructuredGrid(0x7f8b01026810) at 0x1a55e3280>,
 'VA08B_04': <vtkmodules.vtkCommonDataModel.vtkUnstructuredGrid(0x7f8b0072a890) at 0x1a53dbc40>,
 'VA08B_05': <vtkmodules.vtkCommonDataModel.vtkUnstructuredGrid(0x7f8b0072bc50) at 0x1a5417c40>,
 'VA08B_06': <vtkmodules.vtkCommonDataModel.vtkUnstructuredGrid(0x7f8b0072e290) at 0x1a53ba200>,
 'VA08B_07': <vtkmodules.vtkCommonDataModel.vtkUnstructuredGrid(0x7f8b0102a410) at 0x1a55e3fa0>,
 'VA08B_08': <vtkmodules.vtkCommonDataModel.vtkUnstructuredGrid(0x7f8b508a5a60) at 0x1a56242e0>,
 'VA08B_09': <vtkmodules.vtkCommonDataModel.vtkUnstructuredGrid(0x7f8b007283c0) at 0x1a55e3f40>,
 'VA08B_10': <vtkmodules.vtkCommonDataModel.vtkUnstructuredGrid(0x7f8b007339f0) at 0x1a5624340>,
 'VA08B_11': <vtkmodules.vtkCo

### Read SimVascular Pathline Annotations

In [20]:
with open(centerline_file_, "r") as path:
    #path.readlines()
    tree = ET.parse(path)
root = tree.getroot()

direction_points = []
for direction_point in root.findall(".//path_point/tangent"):
    x = float(direction_point.attrib['x'])
    y = float(direction_point.attrib['y'])
    z = float(direction_point.attrib['z'])
    direction_points.append((x,y,z))

position_points = []
for path_point in root.findall(".//path_point/pos"):
    x = float(path_point.attrib['x'])
    y = float(path_point.attrib['y'])
    z = float(path_point.attrib['z'])
    position_points.append((x,y,z))

path_normals = []
for normal in root.findall(".//path_point/rotation"):
    x = float(normal.attrib['x'])
    y = float(normal.attrib['y'])
    z = float(normal.attrib['z'])
    path_normals.append((x,y,z))

NPoints = len(position_points)
print("the number of points in the centerline is:", NPoints)

binormal = []
for i in range(NPoints):
    binormal_ = np.cross(np.array(direction_points[i]),np.array(path_normals[i]))
    binormal_ /= np.linalg.norm(binormal_)
    binormal.append(binormal_)
    

the number of points in the centerline is: 111


### Convert Pathline to VTP

In [19]:
def points_to_vtp(points):
    # Create VTK points
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)

    # Create a polyline
    polyline = vtk.vtkPolyLine()
    polyline.GetPointIds().SetNumberOfIds(len(points))
    for i in range(len(points)):
        polyline.GetPointIds().SetId(i, i)

    # Create a cell array to store the polyline
    cells = vtk.vtkCellArray()
    cells.InsertNextCell(polyline)

    # Create a polydata object
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(vtk_points)
    polydata.SetLines(cells)

    return polydata

In [ ]:
centerline_vtp = points_to_vtp(position_points)
peak_intensity = 5
image_ = ReadVTKFile(Image_names[peak_intensity])

### Search for the Maximum Gradient

In [ ]:
def gradient_filter(vtk_image):
    gradient_filter = vtk.vtkImageGradient()
    gradient_filter.SetInputData(vtk_image)
    gradient_filter.SetDimensionality(3)
    gradient_filter.Update()

    return gradient_filter.GetOutput()
    
def define_borders(gradient_image):
    magnitude_filter = vtk.vtkImageMagnitude()
    magnitude_filter.SetInputData(gradient_image)
    magnitude_filter.Update()
    
    return magnitude_filter.GetOutput()

In [22]:
image_gradient_PI = define_borders(gradient_filter(image_))
centerline_vtp_projected = ProbeFilter(centerline_vtp, image_gradient_PI)
WriteVTPFile(os.path.join(path_, f"{graft_name}.vtp"), centerline_vtp_projected)

In [26]:
magnitude_array_name = "Magnitude"
MagnitudeArray = centerline_vtp_projected.GetPointData().GetArray(magnitude_array_name)
ptidx = np.argmax(MagnitudeArray)
Shuttle_mode_point = centerline_vtp_projected.GetPoint(ptidx)

In [33]:
def clip_polydata_with_plane(polydata, origin, normal, inside_out=False):
    plane = vtk.vtkPlane()
    plane.SetOrigin(origin)
    plane.SetNormal(normal)

    clipper = vtk.vtkClipPolyData()
    clipper.SetInputData(polydata)
    clipper.SetClipFunction(plane)

    if inside_out:
        clipper.InsideOutOn()
    else:
        clipper.InsideOutOff()
    
    clipper.Update()

    return clipper.GetOutput()

def clip_USGrid_w_plane(USGrid, origin, normal, inside_out = False):
    plane = vtk.vtkPlane()
    plane.SetOrigin(origin)
    plane.SetNormal(normal)

    clipper = vtk.vtkClipDataSet()
    clipper.SetInputData(USGrid)
    clipper.SetClipFunction(plane)

    if inside_out:
        clipper.InsideOutOn()
    else:
        clipper.InsideOutOff()
    
    clipper.Update()

    return clipper.GetOutput()

In [38]:
UpperMesh = {}
LowerMesh = {}

for key, mesh in MeshProjections.items():
    upper_mesh = clip_USGrid_w_plane(mesh, position_points[ptidx], direction_points[ptidx], True)
    mesh_name_upper = f"{graft_name}_{key}_upper"
    UpperMesh[key] = upper_mesh
    WriteVTUFile(os.path.join(path_,f"{mesh_name_upper}.vtu"), upper_mesh)
    
    lower_mesh = clip_USGrid_w_plane(mesh, position_points[ptidx], direction_points[ptidx])
    mesh_name_lower = f"{graft_name}_{key}_lower"
    LowerMesh[key] = lower_mesh
    WriteVTUFile(os.path.join(path_,f"{mesh_name_lower}.vtu"), lower_mesh)


centerline_upper = clip_polydata_with_plane(centerline_vtp_projected, position_points[ptidx], direction_points[ptidx], True)
centerline_lower = clip_polydata_with_plane(centerline_vtp_projected, position_points[ptidx], direction_points[ptidx])

### Cutting the Lumen Every 3 mm

In [117]:
cumulative_distance = [0]
for i in range(1, NPoints):
    dist_ = np.sqrt((position_points[i][0] - position_points[i-1][0])**2 +
                    (position_points[i][1] - position_points[i-1][1])**2 +
                    (position_points[i][2] - position_points[i-1][2])**2 )
    cumulative_distance.append(dist_ + cumulative_distance[i-1])


In [118]:
remainder  = np.array([int(i)%3 for i in cumulative_distance])
indeces = np.where(remainder == 0)
indeces = list(indeces[0])
print(indeces)
indeces_ = [indeces[i] - indeces[i-1] for i in range(1, len(indeces))]
print(indeces_)
indeces_1 = [0]
indeces_1.extend(indeces_)
indeces_1 = indeces_1

indeces_2 = [indeces[i] for i in range(len(indeces_1)) if indeces_1[i] != 1]
print(indeces_2)

[0, 1, 2, 3, 4, 9, 13, 17, 21, 22, 26, 27, 31, 32, 36, 40, 41, 45, 46, 50, 51, 55, 59, 60, 64, 69, 73, 74, 78, 79, 83, 89, 92, 96, 101, 105]
[1, 1, 1, 1, 5, 4, 4, 4, 1, 4, 1, 4, 1, 4, 4, 1, 4, 1, 4, 1, 4, 4, 1, 4, 5, 4, 1, 4, 1, 4, 6, 3, 4, 5, 4]
[0, 9, 13, 17, 21, 26, 31, 36, 40, 45, 50, 55, 59, 64, 69, 73, 78, 83, 89, 92, 96, 101, 105]


In [119]:
def slice_w_plane(Volume,Origin,Norm):
    plane=vtk.vtkPlane()
    plane.SetOrigin(Origin)
    plane.SetNormal(Norm)
    Slice=vtk.vtkCutter()
    Slice.GenerateTrianglesOff()
    Slice.SetCutFunction(plane)
    Slice.SetInputData(Volume)
    Slice.Update()
    return Slice.GetOutput()

In [120]:
CrossSections = vtk.vtkAppendPolyData()
for idx in indeces_2:
    cross_section_ = slice_w_plane(MeshProjections['VA08B_06'], position_points[idx+2], direction_points[idx+2])
    CrossSections.AddInputData(cross_section_)
CrossSections.Update()
WriteVTPFile(os.path.join(path_, "CrossSections.vtp"), CrossSections.GetOutput())